# Challenge Questions, Challenge Risk, and Enrich Context Workflow

This notebook demonstrates how to use the challenge_questions, challenge_risk chains, and the enrich_context workflow in RiskGPT.


## 1. Setup and Installation

First, let's install RiskGPT and set up the OpenAI API key.


In [ ]:
# Install RiskGPT directly from GitHub (if needed)
# !pip install git+https://github.com/thwolter/riskgpt.git

# Import required modules
import os
from getpass import getpass
import asyncio

# Prompt for OpenAI API key (this way it won't be visible in the notebook)
openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Optional: Set other configuration variables if needed
os.environ["OPENAI_MODEL_NAME"] = "openai:gpt-4.1-nano"  # Default model
os.environ["MEMORY_TYPE"] = "buffer"  # Default memory type

# Configure logging
from riskgpt.logger import configure_logging
configure_logging()


## 2. Create a Business Context

Let's create a business context that we'll use throughout this notebook.


In [5]:
from riskgpt.models.common import BusinessContext
from riskgpt.models.chains.risk import Risk
from riskgpt.models.enums import AudienceEnum

# Create a business context
context = BusinessContext(
    project_id="CLOUD-2023",
    project_description="Migrate on-premises infrastructure to cloud services",
    domain_knowledge="The company is a financial services provider with strict regulatory requirements",
    business_area="IT Infrastructure",
    industry_sector="Financial Services"
)

# Create a sample risk
sample_risk = Risk(
    title="Data Security Breach",
    description="Risk of unauthorized access to sensitive data during migration",
    category="Security",
)


## 3. Challenge Questions Chain

The challenge_questions_chain generates challenging questions from a business context that can be used for internet searches.


In [6]:
from riskgpt.models.chains.questions import ChallengeQuestionsRequest
from riskgpt.chains.challenge_questions import challenge_questions_chain

# Create a request
questions_request = ChallengeQuestionsRequest(
    business_context=context,
    audience=AudienceEnum.risk_internal,
    focus_areas=["data security", "compliance", "service continuity"],
    num_questions=5
)

# Run the chain
async def run_challenge_questions():
    response = await challenge_questions_chain(questions_request)
    print(f"Generated {len(response.questions)} challenging questions:")
    for i, question in enumerate(response.questions, 1):
        print(f"{i}. {question}")
    return response

# Execute the async function
challenge_questions_response = await run_challenge_questions()


Generated 5 challenging questions:
1. What are the primary data security vulnerabilities associated with migrating sensitive financial data to cloud infrastructure in a highly regulated environment?
2. How can the company ensure compliance with strict regulatory requirements during and after the cloud migration process to prevent legal and financial penalties?
3. What are the potential risks to service continuity during the transition from on-premises to cloud infrastructure, and how can they be mitigated?
4. What challenges might arise in maintaining data integrity and confidentiality when integrating legacy financial systems with cloud services?
5. How can the company proactively identify and address emerging cybersecurity threats specific to cloud environments within the financial sector?


## 4. Challenge Risk Chain

The challenge_risk_chain generates challenging questions for a specific risk to help stakeholders better understand and address the risk.


In [7]:
from riskgpt.models.chains.questions import ChallengeRiskRequest
from riskgpt.chains.challenge_risk import challenge_risk_chain

# Create a request
risk_request = ChallengeRiskRequest(
    risk=sample_risk,
    business_context=context,
    audience=AudienceEnum.risk_internal,
    focus_areas=["data encryption", "access controls", "regulatory compliance"],
    num_questions=3
)

# Run the chain
async def run_challenge_risk():
    response = await challenge_risk_chain(risk_request)
    print(f"Generated {len(response.questions)} challenging questions for risk '{sample_risk.title}':")
    for i, question in enumerate(response.questions, 1):
        print(f"{i}. {question}")
    return response

# Execute the async function
challenge_risk_response = await run_challenge_risk()


Generated 3 challenging questions for risk 'Data Security Breach':
1. How robust are the encryption protocols applied during data transfer and storage, and have they been tested against evolving threat vectors specific to financial data?
2. In what ways are access controls, including identity management and multi-factor authentication, validated to prevent privilege escalation or insider threats during and after migration?
3. Are the compliance and regulatory requirements for data security explicitly integrated into the migration plan, and how are potential gaps in adherence identified and addressed prior to execution?


## 5. Enrich Context Workflow

The enrich_context workflow enriches a business context with external information from various sources (news, professional, regulatory).


In [11]:
from riskgpt.models.workflows.context import EnrichContextRequest
from riskgpt.workflows.enrich_context import enrich_context

# Create a request
enrich_request = EnrichContextRequest(
    business_context=context,
    focus_keywords=["cloud migration", "financial services", "data security"],
    time_horizon_months=12
)

# Run the workflow
async def run_enrich_context():
    response = await enrich_context(enrich_request)
    print("Sector Summary:")
    print(response.sector_summary)
    print("\nWorkshop Recommendations:")
    for i, rec in enumerate(response.workshop_recommendations, 1):
        print(f"{i}. {rec}")
    if response.full_report:
        print("\nFull Report:")
        print(response.full_report[:500] + "..." if len(response.full_report) > 500 else response.full_report)
    return response

# Execute the async function
enrich_context_response = await run_enrich_context()


InvalidUpdateError: At key 'search_failed': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE

In [ ]:
enrich_context_response.full_report

## 6. Combining the Workflows

Now let's see how we can combine these workflows to create a more comprehensive risk analysis.


In [ ]:
async def combined_workflow():
    # Step 1: Enrich the context with external information
    print("Step 1: Enriching context with external information...")
    enrich_response = await enrich_context(enrich_request)
    
    # Step 2: Generate challenging questions based on the enriched context
    print("\nStep 2: Generating challenging questions...")
    questions_response = await challenge_questions_chain(questions_request)
    
    # Step 3: Generate challenging questions for a specific risk
    print("\nStep 3: Generating challenging questions for a specific risk...")
    risk_response = await challenge_risk_chain(risk_request)
    
    # Return all responses
    return {
        "enrich_response": enrich_response,
        "questions_response": questions_response,
        "risk_response": risk_response
    }

# Execute the combined workflow
combined_results = asyncio.run(combined_workflow())

# Display a summary of the results
print("\nSummary of Combined Workflow:")
print(f"- Enriched context with {len(combined_results['enrich_response'].workshop_recommendations)} recommendations")
print(f"- Generated {len(combined_results['questions_response'].questions)} challenging questions from business context")
print(f"- Generated {len(combined_results['risk_response'].questions)} challenging questions for risk '{sample_risk.title}'")


## 7. Conclusion

In this notebook, we've demonstrated how to use the challenge_questions, challenge_risk chains, and the enrich_context workflow in RiskGPT. These tools can help risk managers and stakeholders better understand and address risks in their projects.

Key takeaways:
- The challenge_questions_chain generates questions from a business context for internet searches
- The challenge_risk_chain generates questions for a specific risk to help stakeholders address it
- The enrich_context workflow enriches a business context with external information
- These tools can be combined to create a comprehensive risk analysis workflow